In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
# # Load the train and test sets
# train_df = pd.read_csv('train_data.csv')
# test_df = pd.read_csv('test_data.csv')

# # Combine the two datasets. Set ignore_index to False.
# df = pd.concat([train_df,test_df],ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
# df.to_csv('data.csv',index=False)

In [25]:
df=pd.read_csv("backup.csv")
df =df.sample(10000,random_state=42)
df.to_csv('sample.csv',index=False)
df.head()

,title,link,domain,published_date,topic
27583,Glasgow pub hit by coronavirus as drinker test...,https://in.news.yahoo.com/glasgow-pub-conducts...,in.news.yahoo.com,2020-08-06 14:09:00,HEALTH
94783,Dinosaur leaves mark in Kelantan,https://www.thestar.com.my/news/nation/2020/08...,thestar.com.my,2020-08-16 00:12:11,NATION
101314,"India crosses grim milestone of 50,000 COVID-1...",https://www.geo.tv/latest/303124-india-crosses...,geo.tv,2020-08-17 04:59:00,WORLD
15627,"Tom Brady, Rob Gronkowski, and Buccaneers hit ...",https://www.bostonglobe.com/2020/08/04/sports/...,bostonglobe.com,2020-08-04 23:55:02,SPORTS
2888,Can Top Glove surpass market cap of Asean's la...,https://www.theedgemarkets.com/article/can-top...,theedgemarkets.com,2020-08-03 09:00:00,BUSINESS


In [26]:
df=df.drop(["link","domain","published_date"],axis=1)

In [27]:
#The percentage of each class in the data

df.topic.value_counts(normalize=True).sort_index().mul(100).round(1).astype(str) + '%'

topic
BUSINESS         13.4%
ENTERTAINMENT    14.3%
HEALTH           13.4%
NATION           14.0%
SCIENCE           3.2%
SPORTS           13.6%
TECHNOLOGY       13.8%
WORLD            14.3%
Name: proportion, dtype: object

In [23]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['title'] = df['title'].apply(lower_case)
        df['title'] = df['title'].apply(remove_stop_words)
        df['title'] = df['title'].apply(removing_numbers)
        df['title'] = df['title'].apply(removing_punctuations)
        df['title'] = df['title'].apply(removing_urls)
        df['title'] = df['title'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [28]:
df=normalize_text(df)
df.head()

,title,topic
27583,glasgow pub hit coronavirus drinker test posit...,HEALTH
94783,dinosaur leaf mark kelantan,NATION
101314,india cross grim milestone covid death,WORLD
15627,tom brady rob gronkowski buccaneer hit practic...,SPORTS
2888,top glove surpass market cap asean s largest bank,BUSINESS


In [29]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df["topic"] = label_encoder.fit_transform(df["topic"])

In [45]:
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(df['title'])
y = df['topic']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [31]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/bhauryal7/NewsClassification.mlflow')
dagshub.init(repo_owner='bhauryal7', repo_name='NewsClassification', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

Accessing as bhauryal7

Initialized MLflow to track repo "bhauryal7/NewsClassification"

Repository bhauryal7/NewsClassification initialized!

2025/03/18 16:43:31 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/49127cd8129d4f7da5c8e6ddcf808605', creation_time=1742312611100, experiment_id='0', last_update_time=1742312611100, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [47]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "TfidfVectorizer")
        mlflow.log_param("num_features", 10000)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(C=1,max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)

        class_report = classification_report(y_test,y_pred)

        print("\nClassification Report:\n", class_report)

        # Log metrics in MLflow
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_text(class_report, "classification_report.txt")

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-03-18 16:53:43,555 - INFO - Starting MLflow run...
2025-03-18 16:53:43,776 - INFO - Logging preprocessing parameters...
2025-03-18 16:53:44,452 - INFO - Initializing Logistic Regression model...
2025-03-18 16:53:44,453 - INFO - Fitting the model...
2025-03-18 16:53:44,921 - INFO - Model training complete.
2025-03-18 16:53:44,922 - INFO - Logging model parameters...
2025-03-18 16:53:45,165 - INFO - Making predictions...
2025-03-18 16:53:45,170 - INFO - Calculating evaluation metrics...



Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.62      0.65       259
           1       0.71      0.83      0.77       293
           2       0.70      0.79      0.74       260
           3       0.57      0.60      0.58       273
           4       0.91      0.44      0.59        71
           5       0.88      0.83      0.86       283
           6       0.83      0.79      0.81       294
           7       0.60      0.58      0.59       267

    accuracy                           0.71      2000
   macro avg       0.74      0.69      0.70      2000
weighted avg       0.72      0.71      0.71      2000



2025-03-18 16:53:46,313 - INFO - Saving and logging the model...
2025/03/18 16:53:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-18 16:53:53,483 - INFO - Model training and logging completed in 9.71 seconds.


🏃 View run bemused-roo-725 at: https://dagshub.com/bhauryal7/NewsClassification.mlflow/#/experiments/0/runs/cf3eeb6d63804f338b86ca5d8414e18a
🧪 View experiment at: https://dagshub.com/bhauryal7/NewsClassification.mlflow/#/experiments/0
